In [1]:
import tqdm
import pandas as pd
import requests

In [2]:
headers = {
    'authority': 'wbtc.network',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'referer': 'https://wbtc.network/dashboard/order-book',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}
results = []

params = {
    'limit': '30',
}

while(True):
  print(params)
  response = requests.get('https://wbtc.network/api/chain/eth/token/wbtc/orders', params=params, headers=headers)
  res_json = response.json()
  tmp_results = res_json.get('result', [])
  results.extend(tmp_results)
  if len(tmp_results) == 0:
    break
  else:
    prev_id = res_json['result'][-1]['id']
    params = {
        'limit': '30',
        'idLt': prev_id,
    }

{'limit': '30'}
{'limit': '30', 'idLt': '633720f6bbb6b1ad37a64af97c9dafe2'}
{'limit': '30', 'idLt': '62f3acff180ed16039382d8b12e8a539'}
{'limit': '30', 'idLt': '62cc4f519ebe2c6b433bae80c21b56aa'}
{'limit': '30', 'idLt': '62aa0626657582577a667d185ddb5383'}
{'limit': '30', 'idLt': '627eae5a3ba62ed0c83fe58126296672'}
{'limit': '30', 'idLt': '62389b16211dd2001f4b4e4774c0da0a'}
{'limit': '30', 'idLt': '61eb11f094336e0020e67a0b61efdcd6'}
{'limit': '30', 'idLt': '6196c36b20ba04001f622ed85e072976'}
{'limit': '30', 'idLt': '61688d09cd492d001e4030d1e1156d09'}
{'limit': '30', 'idLt': '6126a427043d7a001e3a9f485bf7a1da'}
{'limit': '30', 'idLt': '60d9de8bfaa10c001f09c52beae83d35'}
{'limit': '30', 'idLt': '60a55f632c0189001ec865e8a59bfc47'}
{'limit': '30', 'idLt': '60824278738749001ea6b3a7c5dba6be'}
{'limit': '30', 'idLt': '6049cf7c474387001ee9fb538b8e8ffa'}
{'limit': '30', 'idLt': '601f652f11b1d4001e37097ac0ffe3ce'}
{'limit': '30', 'idLt': '5ff6f3db04aea0001e67eaa3995ec928'}
{'limit': '30', 'idLt': 

In [6]:
# dump to json 
import json
with open('wbtc_mint_burn.json', 'w') as f:
    json.dump(results, f)

In [3]:
ROUNDING = 100000000.0

In [4]:
results_df = pd.DataFrame(results)
results_df['amount'] = results_df['amount'].astype(int)

In [29]:
results_agg = results_df.groupby(['merchantName', 'requestingMerchant', 'type'])['amount'].sum().reset_index()

merchant_sum = results_agg.pivot(index=['merchantName', 'requestingMerchant'], columns='type', values='amount').div(ROUNDING).fillna(0).round(1)
merchant_sum['net'] = merchant_sum['mint'] - merchant_sum['burn']
merchant_sum.sort_values('net', ascending=False)

,type,burn,mint,net
merchantName,requestingMerchant,,,
Unknown,5e5ec5d8a221fb000946fd6d32bb63f5,1269.2,20002016.1,20000746.9
FalconX,5f975a29b0709b001e4461b52b4e55a1,3700.0,352794.9,349094.9
Unknown,5f06229d39e1af001e2d5fe430c24762,29435.3,111252.9,81817.6
Grapefruit Trading,5f1a2ed50a60ba00081b8e194b8a5c98,6425.4,77321.5,70896.1
CoinList,5e5ec5d8a221fb000946fd70dc114d92,9564.0,50116.5,40552.5
imToken,5f192ed5f35edc001efa28bd9c12fd6a,4959.7,43803.1,38843.4
Three Arrows Capital,5f0602ee39e1af001e2d5fe238e66f68,4671.4,18737.9,14066.5
CoinList,6026c31eb1aa21001e27992316919f0c,57771.2,67724.6,9953.4
Amber,619f619281c4eb001f8de58acd6a5c62,4300.7,8306.1,4005.4


In [14]:
results_df[results_df['merchantName'] == "Unknown"].groupby(['requestingMerchant', 'type'])['amount'].sum()

requestingMerchant                type
5e5ec5d8a221fb000946fd6d32bb63f5  burn        126919696969
                                  mint    2000201609129000
5f06229d39e1af001e2d5fe430c24762  burn       2943528293320
                                  mint      11125294727909
623324d4f5e9ba001f7c457e52495cae  burn              998800
                                  mint              998800
Name: amount, dtype: int64

In [32]:
merchant_id = "5e5ec5d8a221fb000946fd6d32bb63f5"
for res in results:
  if res['requestingMerchant'] == merchant_id:
    for h in res['history']:
      if h['chain'] == 'eth':
        print(h)

{'id': '5f078b6d6cea05001eefa6799b05cce9', 'token': 'wbtc', 'txid': '0x57172ce2411a2c2c2a72deeda4461abf09797c435ee09fb9d69bdd898afd6645', 'chain': 'eth', 'blockHeight': 10427658, 'amount': '26969696969', 'type': 'burn', 'date': '2020-07-09T21:23:09.000Z', 'action': 'pending'}
{'id': '5f07ce7c6de38b001e67bc739d45c923', 'token': 'wbtc', 'txid': '0x567554ccab2edc99c4970e6e70f81f719c6e38819d1d31f9c501f40da03d737f', 'chain': 'eth', 'amount': '26969696969', 'type': 'burn', 'date': '2020-07-10T02:12:12.380Z', 'action': 'completed'}
{'id': '5efa5c79fe30e0001e2ee2d846cdf18c', 'token': 'wbtc', 'txid': '0xa11f21265139312a14f7f1ee493f6811f90c05e112a26756f5a1861606e62734', 'chain': 'eth', 'blockHeight': 10363086, 'amount': '11491950000', 'type': 'mint', 'date': '2020-06-29T21:25:21.000Z', 'action': 'pending'}
{'id': '5efa6e13fe30e0001e2ee2db5899caf2', 'token': 'wbtc', 'txid': '0x1c3ba37a11e4f4dd65bfa9502023e2941b2cfaf4200c0cfcac32703b840e7979', 'chain': 'eth', 'amount': '11491950000', 'type': 'mint

269.69696969